In [1]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install nltk

In [1]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
# import bs4
# %matplotlib inline
# import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
# from nltk.tokenize import TreebankWordTokenizer
# import Levenshtein  # package python-Levenshtein

# Additional imports from A5
import nltk
# nltk.download()
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /Users/Pegah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# SPACY package
# import spacy
# nlp = spacy.load('en')

# Import Dataset

In [3]:
# import the json
import json
with open("data3.json", "r") as f:
    women_summaries = json.load(f)
    
# PEGAH (From A5)
print("Each woman's entry is a dictionary with the following keys...")
print(women_summaries[0].keys())

deduped_women_summaries = {}
for woman in women_summaries:
    if woman['name'] not in deduped_women_summaries:
        deduped_women_summaries[woman['name']] = woman['summary']
    else:
        deduped_women_summaries[woman['name']] += "\n" + woman['summary']

Each woman's entry is a dictionary with the following keys...
dict_keys(['views', 'summary', 'name'])


In [4]:
# REMOVED FROM SPACY VERSION
# creates a list of women to keep as a global variable
# women_names = list()
# for i in range(len(women_summaries)):
#     name = women_summaries[i]['name']
#     end_index = name.find('(')
#     if end_index != -1 and name[: (end_index-1)] not in women_names :
#         women_names.append(name[: (end_index-1)])
#     elif end_index == -1 and name not in women_names :
#         women_names.append(name)
        
# # print(women_names)
# #women_names is a list of the names of women in order of the JSON file

# Deduplicate

In [5]:
deduped_women = [{'summary': summary, 'name': name} for (name, summary) in list(deduped_women_summaries.items())]

# Spacy Dictionary (Commented out for now)

In [6]:
from __future__ import unicode_literals

In [6]:
# # Builds a dictionary that maps woman to the list of tokens associated with her
# # {'woman1' : ['token1', 'token2', ... 'tokenn']}
# from __future__ import unicode_literals

# def build_spacy_token_dictionary(input_summaries, input_women_names):
#     # Builds a dictionary that maps each woman to their tokenized words (yes we've already done this but we want the spacy version)
#     women_token_dictionary = dict()
    
#     for i in range(len(input_women_names)):
#         name = input_women_names[i]
#         summary = input_summaries[i]['summary']
    
#         index_was = summary.find('was ')
#         index_is = summary.find('is ')
#         index_currently = summary.find('currently ')
#         if (index_was == -1 and index_is == -1):
#             summary2 = summary[index_currently:]
#         elif (index_was == -1 and index_currently == -1):
#             summary2 = summary[index_is:]
#         elif (index_is == -1 and index_currently == -1):
#             summary2 = summary[index_was:]
#         else :
#             index_min = min(index_is, index_was)
#             summary2 = summary[index_min:]
            
#         # lowercase long string of summary
#         summary_lower = summary2.lower()
#         # doc object type
#         summary_nlp = nlp(summary_lower)
#         # type list
#         summary_list = [token.text for token in summary_nlp]
#         # type list, but without stop words
#         summary_token_filtered = [w for w in summary_list if w not in stopwords.words('english')]
        
#         # convert to a string separated by spaces
#         summary_str = " ".join(summary_token_filtered) 
#         summary_nlp = nlp(summary_str)
#         women_token_dictionary[name] = summary_nlp
    
# #     print(women_token_dictionary)
#     return women_token_dictionary

In [7]:
# COMMENTED OUT BECAUSE TOKENIZER DOES THIS
# women_token_dictionary = build_spacy_token_dictionary(women_summaries, women_names)
# # print(women_token_dictionary)

# TF-IDF Vectorizer
#### Includes pickling the vectorizer and the TF-IDF matrix

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english", min_df = 10, max_df = 0.95)
matx = vectorizer.fit_transform(map(lambda x: x["summary"], deduped_women))

import pickle
with open("myvectorizer.pickle", "wb+") as f:
    pickle.dump(vectorizer, f)
    pickle.dump(matx, f)

# Cosine Similarity

In [14]:
# query = "is a mother"
# q_vec = vectorizer.transform([query])
# sim_doc_scores = cosine_similarity(q_vec, matx)
# # apply weighting here
# # print(sum(sim_doc_scores.flatten()))
# sim_docs = np.argsort(sim_doc_scores.flatten())[::-1]
# # sim_docs

# JUST KIDDING!! WE DO THIS IN THE FUNCTION BELOW

# Spacy Similarity Matrix

In [31]:
def build_sim_matrix(input_token_dictionary, input_len_names, input_women_names):
    # Builds matrix using spacy that stores each woman's cosine similarity to each other
    
    sim_matrix = np.zeros(shape = (input_len_names, input_len_names))
    
    for woman1 in input_token_dictionary:
        for woman2 in input_token_dictionary:
            woman1_index = input_women_names.index(woman1)
            woman2_index = input_women_names.index(woman2)
#             print(input_token_dictionary[woman1])
#             print(input_token_dictionary[woman2])
            sim_matrix[woman1_index][woman2_index] = input_token_dictionary[woman1].similarity(input_token_dictionary[woman2])
    np.fill_diagonal(sim_matrix, 0)
    
    return sim_matrix
        

In [ ]:
women2women_cosine_sim_matrix = build_sim_matrix(women_token_dictionary, len(women_names), women_names)


In [29]:
# print(women2women_cosine_sim_matrix[1][0])

TypeError: 'NoneType' object has no attribute '__getitem__'

In [25]:
import operator

# Code taken from A5
def get_ranked_women(input_woman, input_sim_matrix, input_women_names) :
#     print(input_sim_matrix)
    '''Return sorted rankings (most to least similar) of women as 
        a list of two-element tuples, where the first element is the 
        woman's name and the second element is the similarity score
    '''
    
    # Get index from woman's name
    idx = input_women_names.index(input_woman)
    
    # Get list of similarity scores for woman
    score_lst = input_sim_matrix[idx]
    women_score_lst = [(input_women_names.index(index), s) for index, score in enumerate(score_lst)]
    
    # Do not account for woman herself in ranking
    women_score_lst = women_score_lst[:idx] + women_score_lst[idx+1:]
    
    # Sort rankings by score (most similar to least similar)
    women_score_lst = sorted(women_score_lst, key=lambda x: -x[1])
    
    # Only returning top 5!
    return women_score_lst[:5]

In [26]:
# Builds a 'top 5 most similar' list of woman for every woman
def create_top_5_dict_women(input_sim_mat, num_women, input_women_names):
    women_top5 = {}
    
    # Loop through each woman
    for woman in range(num_women):
        similarity_list = get_ranked_women(input_women_names[woman], input_sim_mat, input_women_names)
        women_top5[input_women_names[woman]] = similarity_list
        
    return women_top5
            

In [10]:
top5_dict_women2women = create_top_5_dict_women(women2women_cosine_sim_matrix, len(women_names), women_names)
# print(top5_dict_women2women)

NameError: name 'create_top_5_dict_women' is not defined

# Query output for Spacy (Commented out for now)

In [11]:
# SPACY OUTPUT RETURN -- commented out for now
# def return_query(input_query, input_women_token_dictionary):
#     # Given a query string, this function will return a list of women that are most similar to the query
    
#     query = nlp(input_query)
#     results = [] # List that will eventually store name, similarity, popularity, and ultimate RANKING
    
#     # Ranking will be defined by: (1/popularity) + (0.5*similarity) + entity weighting
    
#     for woman in input_women_token_dictionary:
#         similarity = query.similarity(input_women_token_dictionary[woman])
#         if similarity > 0.2: # Arbitrary number, can play around with with
#             results.append((woman, similarity, similarity)) # Currently similarity will act as the ranking
            
#     results.sort(key=lambda x: x[2]) # Ascending order of rankings
#     results.reverse() # Descending order

# Query output for Cossim

In [53]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle

# Load from the pickle
pkl_file = open('myvectorizer.pickle', 'rb')
vectorizer = pickle.load(pkl_file)
matx = pickle.load(pkl_file)

def return_query(vectorizer, matx, query, data_dict):
    q_vec = vectorizer.transform([query])
    sim_doc_scores = cosine_similarity(q_vec, matx)
#     print(sim_doc_scores)
    # Apply weighting here!
    sim_docs = np.argsort(sim_doc_scores.flatten())[::-1]
    
    return_docs = []
    for hit in sim_docs[0:30]:
        return_docs.append(data_dict[hit])
#     print(len(return_docs))
    return return_docs

In [54]:
return_query(vectorizer, matx, "researches cancer", deduped_women)

30


[{'name': 'Sandra Saouaf',
  'summary': 'Sandra Saouaf is an American immunologist who researches autoimmune diseases, such as rheumatoid arthritis, type 1 diabetes, multiple sclerosis and others.'},
 {'name': 'Angela Hartley Brodie',
  'summary': 'Angela Hartley Brodie (28 September 1934 – 7 June 2017) was a British biochemist who pioneered development of steroidal aromatase inhibitors in cancer research. Born in Greater Manchester, Brodie studied chemical pathology to a doctoral level in Sheffield and was awarded a fellowship sponsored by National Institutes of Health. After 17 years of working in Shrewsbury, Massachusetts on oral contraceptives with Harry Brodie, whom she married, she switched focus to the effects of the oestrogen-producing enzyme, aromatase, on breast cancer.\nBrodie managed to get an aromatase inhibitor into a limited clinical trial in breast cancer patients in London, which had such a profound effect that it led to Novartis-sponsored trials, the development of fo